# Collect Images for Training and Testing

In [1]:
import time
import cv2
import openpyxl

In [17]:
labels = ["Victory","ThumbsUp","ThumbsDown"]
paths_train = {            
            "Victory":"D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\Images\\Victory\\",            
            "ThumbsUp": "D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\Images\\ThumbsUp\\",            
            "ThumbsDown": "D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\Images\\Thumbs_Down\\"
        }
paths_test = {
            "Victory":"D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\Images\\Test_Images\\",
            "ThumbsUp": "D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\Images\\Test_Images\\",
            "ThumbsDown": "D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\Images\\Test_Images\\"
}

path = {
    "labelmap":"D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\Images\\labelmap.pbtxt",
    "Train_excel_sheet":"D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\Images\\Data.xlsx",
    "Test_excel_sheet":"D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\Images\\Data1.xlsx",
    "Train_Tfrecord":"D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\TfRecords\\train.record",
    "Test_Tfrecord":"D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\TfRecords\\test.record"
}

number_of_img_train = 10
number_of_img_test = 2

In [3]:
def CaptureImages(Total_Images=None,path=None):
    for lab in labels:
        cap = cv2.VideoCapture(0)
        print("Collecting Images for ",lab)
        time.sleep(8)
        print()
        for i in range(Total_Images):
            print("Image no. ",i+1)
            isPos, frame = cap.read()
            file_name = path[lab]+lab+"{}.jpg".format(str(i+1))
            print(file_name)
            cv2.imshow("Webcam",frame)
            cv2.imwrite(file_name,frame)
            time.sleep(8)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
print("Traing Set")
print()
CaptureImages(number_of_img_train,paths_train)

print("Testing Set")
print()
CaptureImages(number_of_img_test,paths_test)

# Create .pbtxt file

In [6]:
# This function is taking path and creating labelmap.pbtxt file which is required in tfrecords

with open(path["labelmap"], 'w') as lm:
    id = 1
    for lab in labels:
        data = str("item { name : \""+lab+"\" , id : "+str(id)+" }\n")
        lm.write(data)
        id += 1

# Converting XML to EXCEL

In [7]:
import pandas as pd # to create csv file
import xml.etree.ElementTree as ET # to read the xml file
import openpyxl
import os # to identify the file using extension
# Collecting all xml files path from the given directory


def GetAll_XML_FileName(path):
    fileName = []

    for files in os.listdir(path):
        root, ext = os.path.splitext(files)
        if ext == ".xml":
            fileName.append(files)

    return fileName
    

# GetAll_XML_FileName("D:\CODE_Club\Cust_Obj_Detection\Images\ThumbsDown")

#features must be in format : filename,filesource,img-width,img-height,img-depth,xmin,ymin,xmax,ymaxs
def Extract_Features_XML(path):
    features = []
    
    tree = ET.parse(path)
    root = tree.getroot()

    interF = []

    interF.append(root.find('filename').text)
    interF.append(root.find('path').text)
    imgsize = (root.find('size'))
    interF.append(imgsize[0].text)#width
    interF.append(imgsize[1].text)#height
    interF.append(imgsize[2].text)#depth

    for obj in root.findall('object'):
        bnd = obj.find('bndbox')
        x = []
        x.append(bnd[0].text)
        x.append(bnd[1].text)
        x.append(bnd[2].text)
        x.append(bnd[3].text)
        features.append(interF+x)
 
    return features


def Convert_XML_To_CSV(path,labelName):
    all_fileNames = GetAll_XML_FileName(path)
    # First create an .csv file to write
    header = ['FileName','FileSource','Image-Width','Image-Height','Image-Depth','xmin','ymin','xmax','ymax']

    i = 1
    workbk = openpyxl.Workbook()
    wkbSheet = workbk.active

    for data in header:
        wkbSheet.cell(1,i).value = data
        i = i+ 1

    r,c = 2,1
    for eachfile in all_fileNames:
        Path = path+str("\\")+eachfile
        xml_features = (Extract_Features_XML(Path))
        # print(type(xml_features))
        for data in (xml_features):
            for d in data:
                wkbSheet.cell(r,c).value = d
                c = c+ 1
            r = r + 1
            c = 1
        


    wbPath = path + str("\\")+labelName+str(".xlsx")
    workbk.save(wbPath)

In [ ]:
# Call the function this will convert the all xml files to excel file and store it in path speified where xml files are present

# For training 
Convert_XML_To_CSV(paths_train["ThumbsDown"],"ThumbsDown")
Convert_XML_To_CSV(paths_train["ThumbsUp"], "ThumbsUp")
Convert_XML_To_CSV(paths_train["Victory"],"Victory")

#For testing
Convert_XML_To_CSV(paths_test["ThumbsDown"],"ThumbsDown")
Convert_XML_To_CSV(paths_test["ThumbsUp"], "ThumbsUp")
Convert_XML_To_CSV(paths_test["Victory"],"Victory")

# Converting XML to Tfrecords

In [8]:
def labelId(labs):
    j = 1
    for i in labels:
        if i == labs:
            return j
        j = j + 1

In [9]:
import tensorflow as tf

In [10]:
import io
def TF_Record_Creation(gp,fn):
    filename = fn.encode('utf8')
    fileType = "jpg".encode('utf8')
    img_Path = []
    class_Text = []
    class_LbMp = []
    width = [640]
    height = [480]
    minX = []
    minY = []
    maxX = []
    maxY = []

    for i, j in gp.iterrows():
        minX.append(j['xmin'] / width[0])
        maxX.append(j['xmax'] / width[0])
        minY.append(j['ymin'] / height[0])
        maxY.append(j['ymax'] / height[0])
        class_Text.append(j['label'].encode('utf8'))
        class_LbMp.append(labelId(j['label']))
        img_Path.append(j['FileSource'])

    with tf.io.gfile.GFile(img_Path[0],'rb') as fi:
        img = fi.read()

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list = tf.train.Int64List(value=height)),
        'image/width': tf.train.Feature(int64_list= tf.train.Int64List(value=width)),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        # here we give image as input
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[fileType])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=minX)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=maxX)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=minY)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=maxY)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=class_Text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=class_LbMp)),
    }))
    return tf_example
        

In [14]:
def Generate_TfRecords(Excel_Sheet=str,tf_Type=str):
    df = pd.read_excel(Excel_Sheet)
    gk = df.groupby("FileName")
    with tf.io.TFRecordWriter(tf_Type) as file:
        for group_name in gk.groups:
            datafr = gk.get_group(group_name)
            tfRecord = TF_Record_Creation(datafr,group_name)
            SerTF_Record = tfRecord.SerializeToString()
            file.write(SerTF_Record)
        file.close()


In [19]:
# Generate Training 
Generate_TfRecords(path["Train_excel_sheet"],path["Train_Tfrecord"])
Generate_TfRecords(path["Test_excel_sheet"],path["Test_Tfrecord"])

